<a href="https://colab.research.google.com/github/AllergicAlligator/nasa2024/blob/main/20241006_Using_Satsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Suppress library deprecation warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install pystac-client

In [ ]:
!pip install sat-search

In [ ]:
import satsearch
print(satsearch.__version__)

0.3.0


In [ ]:
from pystac_client import Client

LandsatSTAC = Client.open("https://landsatlook.usgs.gov/stac-server", headers=[])

for collection in LandsatSTAC.get_collections():
    print(collection)

<CollectionClient id=landsat-c2l2-sr>
<CollectionClient id=landsat-c2l2-st>
<CollectionClient id=landsat-c2ard-st>
<CollectionClient id=landsat-c2l2alb-bt>
<CollectionClient id=landsat-c2l3-fsca>
<CollectionClient id=landsat-c2ard-bt>
<CollectionClient id=landsat-c2l1>
<CollectionClient id=landsat-c2l3-ba>
<CollectionClient id=landsat-c2l2alb-st>
<CollectionClient id=landsat-c2ard-sr>
<CollectionClient id=landsat-c2l2alb-sr>
<CollectionClient id=landsat-c2l2alb-ta>
<CollectionClient id=landsat-c2l3-dswe>
<CollectionClient id=landsat-c2ard-ta>


In [ ]:
from satsearch import Search

def BuildSquare(lon, lat, delta):
    c1 = [lon + delta, lat + delta]
    c2 = [lon + delta, lat - delta]
    c3 = [lon - delta, lat - delta]
    c4 = [lon - delta, lat + delta]
    geometry = {"type": "Polygon", "coordinates": [[ c1, c2, c3, c4, c1 ]]}
    return geometry

# GPS of Kaohsiung: 22.6273° N, 120.3014° E
geometry = BuildSquare(120.3014, 22.6273, 0.04)

# Searching landsat assets
search = Search.search(
    url='https://landsatlook.usgs.gov/stac-server',
    intersects = geometry,
    # collections=['landsat-8-l1', 'landsat-9-l1'],  # USGS does not use this
    # collections = ["landsat-c2l2-sr"],
    # collections = ["landsat-c2l2-st"],
    collections = ["landsat-c2l1"],
    # collections = ["landsat-c2l2"],
    # query={'landsat:wrs_path': 1, 'landsat:wrs_row': 1},
    datetime='2024-01-01T00:00:00Z/2025-01-01T23:59:59Z',  # Need to use RFC 3339 format
    limit=100
)

In [ ]:
# Save search results
items = search.items()
items.save('subset.geojson')

In [ ]:
# Checking the first five items of the results
for item in items[:5]:
    print(item)

LC08_L1GT_219200_20241001_20241006_02_T2
LC09_L1GT_117045_20240929_20240929_02_T2
LC09_L1TP_117044_20240929_20240929_02_T1
LC08_L1TP_118044_20240928_20241005_02_T1
LC09_L1GT_219200_20240923_20240923_02_T2


In [ ]:
# checking the saved geojson
import geopandas as gpd

gf = gpd.read_file('subset.geojson')
gf.head()

,id,datetime,eo:cloud_cover,view:sun_azimuth,view:sun_elevation,platform,view:off_nadir,landsat:cloud_cover_land,landsat:wrs_type,landsat:wrs_path,...,landsat:correction,accuracy:geometric_x_bias,accuracy:geometric_y_bias,accuracy:geometric_x_stddev,accuracy:geometric_y_stddev,accuracy:geometric_rmse,proj:epsg,created,updated,geometry
0,LC08_L1GT_219200_20241001_20241006_02_T2,2024-10-01 13:53:42.342000+00:00,NaN,-58.093596,-56.717387,LANDSAT_8,0,-1.00,2,219,...,L1GT,NaN,NaN,NaN,NaN,NaN,32651,2024-10-06 02:35:21.832000+00:00,2024-10-06 02:35:21.832000+00:00,"POLYGON ((121.79983 24.15983, 120.01761 23.798..."
1,LC09_L1GT_117045_20240929_20240929_02_T2,2024-09-29 02:21:30.442000+00:00,50.52,136.791703,58.085901,LANDSAT_9,0,20.31,2,117,...,L1GT,NaN,NaN,NaN,NaN,NaN,32651,2024-09-29 06:55:44.453000+00:00,2024-09-29 06:55:44.453000+00:00,"POLYGON ((120.26878 22.71176, 119.87131 20.980..."
2,LC09_L1TP_117044_20240929_20240929_02_T1,2024-09-29 02:21:06.546000+00:00,88.84,138.674729,57.170308,LANDSAT_9,0,77.40,2,117,...,L1TP,0.0,0.0,3.745,3.777,5.319,32651,2024-09-29 06:54:53.594000+00:00,2024-09-29 06:54:53.594000+00:00,"POLYGON ((120.60433 24.15278, 120.20196 22.422..."
3,LC08_L1TP_118044_20240928_20241005_02_T1,2024-09-28 02:27:13.353000+00:00,59.74,138.136241,57.434566,LANDSAT_8,0,40.67,2,118,...,L1TP,0.0,0.0,4.628,4.779,6.652,32650,2024-10-05 17:56:30.584000+00:00,2024-10-05 17:56:30.584000+00:00,"POLYGON ((119.08717 24.15488, 118.68502 22.424..."
4,LC09_L1GT_219200_20240923_20240923_02_T2,2024-09-23 13:53:36.903000+00:00,NaN,-54.653542,-54.200530,LANDSAT_9,0,-1.00,2,219,...,L1GT,NaN,NaN,NaN,NaN,NaN,32651,2024-09-23 15:21:58.258000+00:00,2024-09-23 15:21:58.258000+00:00,"POLYGON ((121.81548 24.16022, 120.02799 23.795..."


In [ ]:
# checking what's in the dataframe
gf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   id                           111 non-null    object             
 1   datetime                     111 non-null    datetime64[ms, UTC]
 2   eo:cloud_cover               104 non-null    float64            
 3   view:sun_azimuth             111 non-null    float64            
 4   view:sun_elevation           111 non-null    float64            
 5   platform                     111 non-null    object             
 6   view:off_nadir               111 non-null    int32              
 7   landsat:cloud_cover_land     111 non-null    float64            
 8   landsat:wrs_type             111 non-null    object             
 9   landsat:wrs_path             111 non-null    object             
 10  landsat:wrs_row              111 non-null 